In [ ]:
import pandas as pd

df = pd.read_csv("online_retail_II.csv",encoding="ISO-8859-1")

print(f"Original Row Count: {len(df)}")


# Remove rows with missing Customer ID
df = df.dropna(subset=["Customer ID"])

# Remove rows with non-positive Quantity and  bad Price
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)]

# Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

df["TotalAmount"] = df["Quantity"] * df["Price"]

print(f"Cleaned Row Count: {len(df)}")
df.head()



import sqlite3

conn = sqlite3.connect("retail.db")


df.to_sql("transactions", conn, if_exists="replace", index=False)

print("Data successfully loaded into 'retail.db' SQL database.")



max_date = df["InvoiceDate"].max()
print(f"Max Invoice Date in Data: {max_date}")

query = """
SELECT
    "Customer ID" AS CustomerID,
    MAX(InvoiceDate) as LastDate,
    CAST(julianday('2011-12-10') - julianday(MAX(InvoiceDate)) AS INTEGER) AS Recency,
    COUNT(DISTINCT Invoice) AS Frequency,
    SUM(TotalAmount) AS Monetary
FROM transactions
GROUP BY "Customer ID"
HAVING Recency IS NOT NULL
"""
rfm_df = pd.read_sql_query(query, conn)

rfm_df["LastDate"] = pd.to_datetime(rfm_df["LastDate"]).dt.date

print ("RFM Table Created  with Dates:")



from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans


scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_df[["Recency", "Frequency", "Monetary"]])


kmeans = KMeans(n_clusters=4, random_state=42)
rfm_df["Cluster"] = kmeans.fit_predict(rfm_scaled)
d
print(rfm_df.head())


rfm_df.to_csv("customer_segments_final.csv", index=False)

print("✅ File fixed! Now go refresh Power BI.")



Original Row Count: 1067371
Cleaned Row Count: 805549
Data successfully loaded into 'retail.db' SQL database.
Max Invoice Date in Data: 2011-12-09 12:50:00
RFM Table Created  with Dates:
            Recency   Frequency       Monetary
Cluster                                       
0        461.651221    2.212755     765.377939
1         65.996877    7.494926    3132.376644
2         20.538462  131.884615  116621.847769
3          0.500000  148.000000  568712.085000
Project Complete! File 'customer_segments_final1.csv' created successfully.
